In [2]:
## use conda create -n geo_spark apache-sedona[spark] jupyter_notebook
import os
# Set Hadoop environment variables BEFORE importing Spark
os.environ['HADOOP_HOME'] = r'C:\hadoop'
os.environ['PATH'] = os.environ['PATH'] + r';C:\hadoop\bin'
# os.environ['JAVA_HOME'] = r'path_to_your_java_installation'  # Make sure this points to your Java installation

from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator

# # Initialize Spark Session
# spark = SparkSession.builder \
#     .appName("Sedona_Local") \
#     .master("local[*]") \
#     .config("spark.jars.packages", "org.apache.sedona:sedona-python-adapter-3.0_2.12:1.4.0,org.datasyslab:geotools-wrapper:1.4.0-28.2") \
#     .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
#     .config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
#     .getOrCreate()
spark = SparkSession.builder \
    .appName("Sedona_Local") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.jars.packages", "org.apache.sedona:sedona-python-adapter-3.0_2.12:1.4.0,org.datasyslab:geotools-wrapper:1.4.0-28.2") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
    .getOrCreate()

# Register Sedona functions
SedonaRegistrator.registerAll(spark)
# Test Sedona
df = spark.sql("SELECT ST_Point(1.0, 2.0) AS point")
df.show(truncate=False)

+-----------+
|point      |
+-----------+
|POINT (1 2)|
+-----------+



C:\Users\es422\AppData\Local\Temp\ipykernel_32192\739727030.py:30: DeprecationWarning: Call to deprecated function registerAll (Deprecated since 1.4.1, use SedonaContext.create() instead.).
  SedonaRegistrator.registerAll(spark)


In [8]:
from sedona.sql import ST_GeomFromWKT, ST_GeomFromWKB
# Load the Parquet file
df = spark.read.parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\NLSDB\2025_03-03_NLSDB.gdb\Case_03032025_1240.parquet")
# df.select("Shape").show(5, truncate=False)
filtered_df = df.filter(df.CSE_DISP != "Closed")
filtered_df = filtered_df.withColumn("geom", ST_GeomFromWKB(df["Shape"]))
# # Filter CASE_STATUS_nlsdb != 'Done'
# filtered_df = df.filter(df.CSE_DISP != "Closed")
# # filtered_df.count()
# # Convert WKT/WKB/GeoJSON to Sedona Geometry (Assuming you have a WKT column named 'geometry')
# filtered_df = filtered_df.withColumn("geom", ST_GeomFromWKT(df["Shape"]))

# Register DataFrame as a Temp Table for SQL Queries
filtered_df.createOrReplaceTempView("polygons")

# Find Overlapping Polygons using Sedona SQL
overlapping_polygons = spark.sql("""
    SELECT a.*
    FROM polygons a
    JOIN polygons b
    ON ST_Overlaps(a.geom, b.geom)
    WHERE a.id != b.id
""")

# Show Results
overlapping_polygons.count()

Py4JJavaError: An error occurred while calling o138.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 20.0 failed 1 times, most recent failure: Lost task 4.0 in stage 20.0 (TID 117) (laptop executor driver): org.locationtech.jts.geom.TopologyException: side location conflict [ (-116.650800861, 36.72849011000005, NaN) ]
	at org.locationtech.jts.geomgraph.EdgeEndStar.propagateSideLabels(EdgeEndStar.java:289)
	at org.locationtech.jts.geomgraph.EdgeEndStar.computeLabelling(EdgeEndStar.java:125)
	at org.locationtech.jts.operation.relate.RelateComputer.labelNodeEdges(RelateComputer.java:325)
	at org.locationtech.jts.operation.relate.RelateComputer.computeIM(RelateComputer.java:125)
	at org.locationtech.jts.operation.relate.RelateOp.getIntersectionMatrix(RelateOp.java:109)
	at org.locationtech.jts.operation.relate.RelateOp.relate(RelateOp.java:54)
	at org.locationtech.jts.geom.Geometry.relate(Geometry.java:1035)
	at org.locationtech.jts.geom.G